In [1]:
import os

import numpy as np #데이터 처리
import pandas as pd #데이터 처리
import warnings
warnings.filterwarnings('ignore')
from collections import Counter # count 용도

import matplotlib.pyplot as plt # 시각화
import seaborn as sns #시각화

import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용

import random #데이터 샘플링
from sklearn.model_selection import GridSearchCV #모델링
from sklearn.ensemble import RandomForestRegressor #모델링
path = "Jupyter Temp/프로젝트/"
pd.set_option("display.max_rows",None)

In [2]:
import folium

In [3]:
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

# 1. date 변수 변환


In [4]:
train.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,...,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,...,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,...,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
inout = train[["bus_route_id","in_out","latitude","longitude","station_name"]]
inout.drop_duplicates(inplace=True)
inout.head()

,bus_route_id,in_out,latitude,longitude,station_name
0,4270000,시외,33.48990,126.49373,제주썬호텔
1,4270000,시외,33.48944,126.48508,한라병원
2,4270000,시외,33.48181,126.47352,정존마을
3,4270000,시내,33.50577,126.49252,제주국제공항(600번)
4,4270000,시내,33.25579,126.41260,중문관광단지입구


In [6]:
inout = inout.groupby(by=["station_name",'bus_route_id',"in_out"]).mean().reset_index()

In [7]:
inout.head()

,station_name,bus_route_id,in_out,latitude,longitude
0,(구)구판장,30850000,시내,33.329680,126.798050
1,(구)구판장,30860000,시내,33.329705,126.797965
2,(구)구판장,30870000,시내,33.329680,126.798050
3,(구)중문동주민센터,22540000,시내,33.251085,126.434300
4,(구)중문동주민센터,22550000,시내,33.251085,126.434300


In [8]:
jeju = folium.Map([33.4,126.55], zoom_start = 11) 

In [9]:
inout.sort_values(by=["bus_route_id","latitude","longitude"],inplace=True)

In [10]:
inout.reset_index(drop=True,inplace=True)
inout.head()

,station_name,bus_route_id,in_out,latitude,longitude
0,켄싱턴리조트악근천,4270000,시내,33.23440,126.49043
1,왕대왓,4270000,시내,33.23465,126.48241
2,서건도,4270000,시외,33.23595,126.50491
3,강정농협,4270000,시외,33.23603,126.47827
4,대포항,4270000,시외,33.23974,126.43773


In [11]:
inout_loc = inout[["latitude", "longitude"]].values.tolist() 
inout_name = inout["station_name"].values.tolist() 

In [12]:
inout_loc[:6]

[[33.2344, 126.49043],
 [33.23465, 126.48241000000002],
 [33.23595, 126.50491],
 [33.23603, 126.47827],
 [33.239740000000005, 126.43773],
 [33.23977, 126.5645]]

In [13]:
for i in range(len(inout)):
    j = i+1
    
    try:
        if inout["bus_route_id"][i] == inout["bus_route_id"][j]:
            folium.PolyLine(locations=[inout_loc[i], inout_loc[j]], color="green", smooth_factor =2).add_to(jeju)
            
    except:
        print("!")

!


In [14]:
import plotly
from plotly import version
print (version)
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)



import plotly.graph_objects as go
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))

plotly.offline.plot(fig, filename = 'plot')

<module 'plotly.version' from 'C:\\Users\\sue_b\\anaconda3\\lib\\site-packages\\plotly\\version.py'>


'plot.html'

In [15]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

username= "sue-bin"
api_key = "k3VTgwYM3XOGJndThXBC"

chart_studio.tools.set_credentials_file(username=username, api_key = api_key)
py.plot(fig, filename="plot", auto_open=False)


'https://plotly.com/~sue-bin/1/'

In [16]:
inout.head()

,station_name,bus_route_id,in_out,latitude,longitude
0,켄싱턴리조트악근천,4270000,시내,33.23440,126.49043
1,왕대왓,4270000,시내,33.23465,126.48241
2,서건도,4270000,시외,33.23595,126.50491
3,강정농협,4270000,시외,33.23603,126.47827
4,대포항,4270000,시외,33.23974,126.43773


In [17]:
inout_temp = inout[inout["bus_route_id"]==4270000]

In [18]:
inout_temp

,station_name,bus_route_id,in_out,latitude,longitude
0,켄싱턴리조트악근천,4270000,시내,33.234400,126.490430
1,왕대왓,4270000,시내,33.234650,126.482410
2,서건도,4270000,시외,33.235950,126.504910
3,강정농협,4270000,시외,33.236030,126.478270
4,대포항,4270000,시외,33.239740,126.437730
5,서귀포항,4270000,시내,33.239770,126.564500
6,배튼개입구,4270000,시내,33.240910,126.443240
7,월평마을,4270000,시외,33.242730,126.460960
8,약천사,4270000,시외,33.242890,126.450840
9,제주국제컨벤션센터중문대포해안주상절리대,4270000,시외,33.243090,126.424720


In [19]:
inout_temp.sort_values(by=["latitude","longitude"],inplace=True)
inout_temp.head()

,station_name,bus_route_id,in_out,latitude,longitude
0,켄싱턴리조트악근천,4270000,시내,33.23440,126.49043
1,왕대왓,4270000,시내,33.23465,126.48241
2,서건도,4270000,시외,33.23595,126.50491
3,강정농협,4270000,시외,33.23603,126.47827
4,대포항,4270000,시외,33.23974,126.43773


In [20]:
def color(x):
    if inout_temp.iloc[i,2] == "시내":
        return 'rgb(0, 255, 0)'
    else:
        return 'rgb(255, 0, 0)'

In [21]:
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import pandas as pd
fig = go.Figure(go.Scattermapbox())
    
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = inout_temp["longitude"],
    lat = inout_temp["latitude"], 
    marker = {'size': 6},
    line = dict(width = 1),
    hovertext = inout_temp['station_name']
))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'style': "stamen-terrain",
         'center': {'lon': inout_temp["longitude"].mean(), 'lat': inout_temp["latitude"].mean()},
        'zoom': 9},
    showlegend = False
)


py.plot(fig,filename="please", auto_open=False) 

'https://plotly.com/~sue-bin/4/'

In [22]:
flight_paths = []
for i in range(len(df_flight_paths)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df['start_lon'][i], df_flight_paths['end_lon'][i+1]],
            lat = [df_flight_paths['start_lat'][i], df_flight_paths['end_lat'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
        )
    )

fig.update_layout(
    title_text = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
    showlegend = False,
    geo = dict(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

py.iplot(fig,filename="please")

NameError: name 'df_flight_paths' is not defined

In [ ]:
##################### Add surface trace
fig.add_trace(go.Heatmap(z=df.values.tolist(), colorscale="Viridis"))

# Update plot sizing


# Add dropdowns
button_layer_1_height = 1.08
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["colorscale", "Viridis"],
                    label="Viridis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Cividis"],
                    label="Cividis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Blues"],
                    label="Blues",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Greens"],
                    label="Greens",
                    method="restyle"
                ),
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=["reversescale", False],
                    label="False",
                    method="restyle"
                ),
                dict(
                    args=["reversescale", True],
                    label="True",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.37,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=[{"contours.showlines": False, "type": "contour"}],
                    label="Hide lines",
                    method="restyle"
                ),
                dict(
                    args=[{"contours.showlines": True, "type": "contour"}],
                    label="Show lines",
                    method="restyle"
                ),
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.58,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
    annotations=[
        dict(text="colorscale", x=0, xref="paper", y=1.06, yref="paper",
                             align="left", showarrow=False),
        dict(text="Reverse<br>Colorscale", x=0.25, xref="paper", y=1.07,
                             yref="paper", showarrow=False),
        dict(text="Lines", x=0.54, xref="paper", y=1.06, yref="paper",
                             showarrow=False)
    ])

fig.show()

In [ ]:
from plotly.exporess as px

fig= px.scatter_mapbox()

In [23]:
bts = pd.read_csv(path + "bus_bts.csv")

In [24]:
bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1


In [25]:
bts.dtypes

user_card_id           float64
bus_route_id             int64
vhc_id                   int64
geton_date              object
geton_time              object
geton_station_code       int64
geton_station_name      object
getoff_date             object
getoff_time             object
getoff_station_code    float64
getoff_station_name     object
user_category            int64
user_count               int64
dtype: object

In [26]:
from datetime import datetime

def on2date(z):
    try:
        date = z[0]
        time = z[1]
        string = date+" "+time
        final = datetime.strptime(string, "%Y-%m-%d %H:%M:%S")
    except:
        final = np.nan
    return final

def off2date(z):
    try:
        date = z[0]
        time = z[1]
        string = date+" "+time
        final = datetime.strptime(string, "%Y-%m-%d %H:%M:%S")
    except:
        final = np.nan
    return final

In [27]:
bts["geton"] = list(zip(bts["geton_date"],bts["geton_time"]))
bts["getoff"] = list(zip(bts["getoff_date"],bts["getoff_time"]))

bts["geton"] = bts["geton"].apply(lambda x : on2date(x))
bts["getoff"] = bts["getoff"].apply(lambda x : off2date(x))

In [28]:
bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count,geton,getoff
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1,2019-09-10 06:34:45,2019-09-10 07:10:31
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1,2019-09-10 06:34:58,2019-09-10 06:56:27
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1,2019-09-10 07:19:07,2019-09-10 07:40:29
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1,2019-09-09 09:14:47,2019-09-09 10:02:46
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1,2019-09-09 09:28:53,2019-09-09 10:21:37


In [29]:
bts["onbus"] = list(zip(bts["geton"],bts["getoff"]))
bts["onbus"] = bts["onbus"].apply(lambda x : x[1] - x[0])

In [30]:
bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count,geton,getoff,onbus
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1,2019-09-10 06:34:45,2019-09-10 07:10:31,0 days 00:35:46
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1,2019-09-10 06:34:58,2019-09-10 06:56:27,0 days 00:21:29
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1,2019-09-10 07:19:07,2019-09-10 07:40:29,0 days 00:21:22
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1,2019-09-09 09:14:47,2019-09-09 10:02:46,0 days 00:47:59
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1,2019-09-09 09:28:53,2019-09-09 10:21:37,0 days 00:52:44


In [32]:
#출근시간대 버스에 가장 오래/적게 탄 사람
print(min(bts["onbus"]))
print(max(bts["onbus"]))

0 days 00:00:06
0 days 03:59:46


In [33]:
#출근시간대 버스를 가장 여러번 탄사람
bts.groupby(by="user_card_id")["bus_route_id"].count().max()

184

In [34]:
#출근시간대 사람들이 가장 많이 탄 버스정류장
bts.groupby(by="geton_station_name")["user_count"].sum().sort_values(ascending=False).head()

geton_station_name
한라병원          50854
제주시외버스터미널     38024
제주버스터미널       32344
제주도청신제주로터리    31606
제주시청(아라방면)    30177
Name: user_count, dtype: int64

In [35]:
#출근시간때 탄 사람들이 가장 많이 내린 버스정류장
bts.groupby(by="getoff_station_name")["user_count"].sum().sort_values(ascending=False).head()

getoff_station_name
제주국제공항(종점)     44500
한라병원           38637
제주버스터미널        34575
제주시청(아라방면)     25979
제주버스터미널(종점)    23661
Name: user_count, dtype: int64

In [36]:
    #출근시간대 버스의 수
len(bts['vhc_id'].unique())

876

In [37]:
#"학교"가 들어간 버스정류장 수
print(train["station_name"][train["station_name"].str.contains("학교")].unique())
print(len(train["station_name"][train["station_name"].str.contains("학교")].unique()))

['강정초등학교' '추자중학교' '삼양초등학교' '신촌초등학교' '오현중고등학교' '김녕환승정류장(김녕초등학교)'
 '신흥리(마을입구)함덕고등학교' '외도초등학교' '한림고등학교' '애월고등학교' '중문고등학교' '하귀초등학교' '수산초등학교'
 '제주대학교병원' '제주대학교입구' '제주중앙여자고등학교(아라방면)' '제주여자중고등학교(아라방면)'
 '제주여자중고등학교(광양방면)' '표선고등학교' '의귀환승정류장(의귀초등학교)' '월랑마을남녕고등학교' '한국국제학교'
 '남녕고등학교' '보성초등학교' '제주국제대학교' '서귀포여자중학교' '서귀포산업과학고등학교' '제주중앙여자고등학교(광양방면)'
 '제주여자상업고등학교' '오라초등학교' '제주고등학교/중흥S클래스' '한라중학교/부영아파트' '신제주초등학교' '삼성여자고등학교'
 '인화초등학교' '송당초등학교' '표선중학교' '아라초등학교' '제주고등학교' '제주제일고등학교' '노형초등학교' '덕수초등학교'
 '안덕초등학교' '대흘초등학교' '봉개초등학교' '평대초등학교' '제주대학교' '제주한라대학교' '애월중학교' '제주여자중고등학교'
 '한라초등학교' '제주여자중고등학교제주지방해양경찰청' '서귀포여자고등학교' '법호촌온성학교' '중문초등학교' '광령초등학교'
 '한림여자중학교' '제주중학교/제주향교' '제주중앙여자중학교' '더럭초등학교' '제주대학교교육대학' '제주동중학교' '화북초등학교'
 '제주동초등학교' '도련초등학교' '삼화초등학교' '삼성초등학교' '백록초등학교' '도평초등학교' '제주영지학교' '탐라중학교'
 '오름중학교' '월랑초등학교' '제주서중학교' '동광초등학교' '대기고등학교' '신성여자중고등학교' '제주중앙고등학교'
 '이도초등학교' '제주한라대학교(종점)' '제주제일중학교' '남광초등학교' '화북초등학교후문' '제주동여자중학교'
 '제주사대부설중학교' '도리초등학교' '도리초등학교(종점)' '효돈중학교' '효돈초등학교' '위미초등학교' '법환초등학교'
 '서귀포중학교' '서귀북초등학교'

In [38]:
#"병원"이 들어간 버스정류장 수
print(train["station_name"][train["station_name"].str.contains("병원")].unique())
print(len(train["station_name"][train["station_name"].str.contains("병원")].unique()))

['한라병원' '제주대학교병원' '한국병원' '열린병원' '한마음병원' 'S중앙병원' '이호동물병원' '새소망요양병원']
8


In [39]:
#"의원"가 들어간 버스정류장 수
print(train["station_name"][train["station_name"].str.contains("의원")].unique())
print(len(train["station_name"][train["station_name"].str.contains("의원")].unique()))

['한의원' '연동365의원' '911의원' '구한경의원앞']
4


In [40]:
#"의원"가 들어간 버스정류장 수
print(train["station_name"][train["station_name"].str.contains("사무소")].unique())
print(len(train["station_name"][train["station_name"].str.contains("사무소")].unique()))

['대서리사무소' '조천환승정류장(조천리사무소)' '남원환승정류장(남원읍사무소)' '표선환승정류장(표선리사무소)'
 '대정환승정류장(대정읍사무소)' '신창환승정류장(한경면사무소)' '표선환승정류장(표선면사무소)' '성읍환승정류장(성읍1리사무소)'
 '사계리사무소' '안덕면사무소' '성산읍사무소' '송당리사무소' '애월읍사무소' '고성1리사무소' '광령1리사무소' '장전리사무소'
 '납읍리사무소' '조천읍사무소' '수산리사무소앞' '수산리사무소' '위미1리사무소' '남원읍사무소' '남원1리사무소'
 '신풍리사무소' '삼달1리사무소' '토산1리리사무소' '태흥3리사무소' '대평리사무소' '상창리사무소' '일과2리사무소'
 '한경면사무소' '두모리사무소' '유수암리사무소' '소길리사무소' '봉성리사무소' '어음2리사무소' '평대리사무소' '시흥리사무소'
 '세화2리사무소' '무릉1리리사무소앞' '대정읍사무소' '안성리사무소앞' '어음1리사무소' '신흥2리사무소' '난산리사무소'
 '토산1리리사무소앞' '광령3리사무소' '광령2리사무소' '상천리사무소' '한남리사무소' '대림리사무소' '신흥1리사무소'
 '삼달2리사무소' '가시리사무소' '명월리사무소' '수산2리사무소' '추자면사무소']
57


In [41]:
#"의원"가 들어간 버스정류장 수
print(train["station_name"][train["station_name"].str.contains("공항")].unique())
print(len(train["station_name"][train["station_name"].str.contains("공항")].unique()))

['제주국제공항(600번)' '제주국제공항(종점)' '제주국제공항(평화로, 800번)' '제주국제공항(일주동로,516도로)'
 '제주국제공항(대정,화순,일주서로)' '제주국제공항' '제주국제공항입구' '제주국제공항(구제주방면)' '제주국제공항(신제주방면)'
 '공항초소']
10
